In [2]:
import nltk
from nltk import ngrams
from collections import Counter

In [3]:
filename = 'train.txt'
with open(filename,encoding='utf-8') as f:
    lines = f.readlines()
print(len(lines))

16382


In [4]:
# tính 1-gram và tần suất
# tính V (số lượng từ khác nhau)
counter_unigram=Counter()
all_tokens_count=0
for line in lines:
    tokens = nltk.word_tokenize(line.lower())
    if len(tokens)==0:
        continue 
    tokens = ['<s>'] + tokens
    all_tokens_count+=len(tokens)
    counter_unigram.update(tokens)
    
print('all_tokens_count=',all_tokens_count)
V=len(counter_unigram)
print('V =',V)
print(counter_unigram.most_common(10))


all_tokens_count= 321720
V = 16681
[(',', 19071), ('<s>', 16328), ('the', 13061), ('and', 9306), ('to', 8222), ('of', 7082), ('a', 6577), ('that', 5681), ('i', 5302), ('in', 4981)]


In [5]:
# tính 2-gram và tần suất 
counter_bigram = Counter()
for sent in lines:
    tokens = nltk.word_tokenize(sent.lower())
    tokens = ['<s>'] + tokens
    gram2=ngrams(tokens,2)
    join_grams = ['_'.join(list(gram)) for gram in gram2]
    counter_bigram.update(join_grams)

print(len(counter_bigram))
#in thử một số 
print(counter_bigram.most_common(10))

111439
[(',_and', 2308), ('<s>_and', 2169), ('of_the', 1363), ('in_the', 1357), ("it_'s", 1205), ('<s>_i', 1178), ('<s>_so', 1120), (',_i', 977), (',_the', 965), ('<s>_it', 885)]


In [6]:
# tính 3-gram và tần suất 
counter_trigram = Counter()
for sent in lines:
    tokens = nltk.word_tokenize(sent.lower())
    tokens = ['<s>'] + tokens
    gram3=ngrams(tokens,3)
    join_grams = ['_'.join(list(gram)) for gram in gram3]
    counter_trigram.update(join_grams)

print(len(counter_trigram))
#in thử một số 
print(counter_trigram.most_common(10))

210144
[("<s>_it_'s", 428), (',_and_i', 274), ('<s>_now_,', 257), ('<s>_and_i', 254), (",_it_'s", 223), ('<s>_thank_you', 203), ('<s>_this_is', 189), ('said_,_``', 169), ("it_'s_a", 167), ('a_lot_of', 155)]


In [7]:
# tính theo Laplace 
anpha = 0.001
def uni_prob(word):
    return max(1,counter_unigram[word])/all_tokens_count

def bi_prob(word1,word2):
    bi_gram = '_'.join([word1,word2])
    return (counter_bigram[bi_gram]+anpha)/(counter_unigram[word1]+V*anpha)  

def tri_prob(word1,word2,word3):
    bi_gram = '_'.join([word1,word2])
    tri_gram = '_'.join([word1,word2,word3])
    return (counter_trigram[tri_gram]+anpha)/(counter_bigram[bi_gram]+V*anpha)

In [8]:
# Backoff
# tính prob theo từng loại: 1-gram, 2-gram, 3-gram
def uni_prob(word):
    return max(1,counter_unigram[word])/all_tokens_count

def bi_prob(word1,word2):
    bi_gram = '_'.join([word1,word2])
    if counter_bigram[bi_gram]>0:
        return counter_bigram[bi_gram]/counter_unigram[word1]
    else:
        return 0.4*uni_prob(word2)
    
def tri_prob(word1,word2,word3):
    bi_gram = '_'.join([word1,word2])
    tri_gram = '_'.join([word1,word2,word3])
    if counter_trigram[tri_gram]>0:
        return counter_trigram[tri_gram]/counter_bigram[bi_gram]
    else:
        return 0.4*bi_prob(word2,word3)

In [9]:
# laplace smoothing
# tính xác suất của một câu, normalize theo 1 từ 
def probLM(sent,n):
    if n>3 or n<1: # không xét trường hợp này 
        return 0
    tokens=nltk.word_tokenize(sent.lower())
    tokens = ['<s>']+tokens
    prob=1
    for i in range(1,len(tokens)):
        if n==1:
            prob*=uni_prob(tokens[i])
        elif n==2:
            prob*=bi_prob(tokens[i-1],tokens[i])
        elif n==3:
            if i>=2:
                prob*=tri_prob(tokens[i-2],tokens[i-1],tokens[i])
            else:
                prob*=bi_prob(tokens[i-1],tokens[i])
    size = len(tokens)-1
    return prob**(1/size)

In [10]:
# tính xác suất và Perplexity cho một câu Test 
# 1-gram
# 2-gram
# 3-gram 
sent='the human body with new abilities is no longer a question'
#sent='A few years back I to to from'
print('n=1')
pr=probLM(sent,1)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=2')
pr=probLM(sent,2)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=3')
pr=probLM(sent,3)
print('prob=',pr)
print('perplexity=',1/pr)

n=1
prob= 0.0014228645509824066
perplexity= 702.8075858025679
n=2
prob= 0.01602336610840924
perplexity= 62.408859239332294
n=3
prob= 0.147231919138491
perplexity= 6.792005469000022


In [11]:
sent1='the human body with new abilities is no longer a question'
sent2='the human body with knew abilities is know longer a question'
pr=probLM(sent1,3)
print('prob=',pr)
print('perplexity=',1/pr)

pr=probLM(sent2,3)
print('prob=',pr)
print('perplexity=',1/pr)

prob= 0.147231919138491
perplexity= 6.792005469000022
prob= 0.0036709318692291744
perplexity= 272.410394859761


In [13]:
import nltk
import re
def getWord():
    word = []
    filename = 'train.txt'
    with open(filename, encoding="utf8") as f:
        lines = f.readlines()
        #print(lines)
        
    for sent in lines:
        words = nltk.word_tokenize(sent)
    
        for x in words:
            word.append(x)

    return word

def preWord(text,n):
    xr = ''
    probTemp = 0
    words =[]
    words = getWord(); 
    while n > 0:
        for x in words:
            if probLM(text+x,1) > probTemp:
                probTemp = probLM(text+x,2)
                xr = x
        text += xr
    
        print(probTemp)    
                    #print(probLM(text+x,2)
                    
        n-=1
    return text
print(preWord("the human",2))

0.019884178609611593
0.019884178609611593
the human,,
